In [ ]:
pip install imutils

Loading dataset

In [ ]:
from imutils import paths
import cv2
imagePaths = list(paths.list_images("c:/archive/AFDB_face_dataset/AFDB_face_dataset"))
# Path for Ijaz - C:\Users\ijazb\Downloads\Dataset\AFDB_face_dataset

Pre-processing techniques

1.   Contrast adjustment
2.   Bilateral filter

1.   Histogram equalisation

In [ ]:
for imagePath in imagePaths: 
   
   # applying contrast adjustment
 
    image = cv2.imread(imagePath)
    alpha = 1.5
    beta = 0
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
  # applying bilateral filter
 
    bilateral = cv2.bilateralFilter(adjusted,15,75,75) 
 
  # applying histogram equalization
 
    R, G, B = cv2.split(bilateral)
    output1_R = cv2.equalizeHist(R)
    output1_G = cv2.equalizeHist(G)
    output1_B = cv2.equalizeHist(B)
    equ = cv2.merge((output1_R, output1_G, output1_B))
 
    cv2.imwrite(imagePath,equ) 
    cv2.waitKey(0)
cv2.destroyAllWindows()

Face Detection and extraction

In [ ]:
# function to detect and extract face(s)
 
def detect_faces(f_cascade, colored_img, scaleFactor = 1.2):
 
    img_copy = np.copy(colored_img)
    gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY) # convert to grayscale
    faces = f_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=5);
    for (x, y, w, h) in faces: # iterating through each detected face
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)   # drawing a rectangle on the detected face    
    return img_copy
 
# loading LBP cascade classifier
 
lbp_face_cascade = cv2.CascadeClassifier(r'c:\Users\user\anaconda3\Library\etc\lbpcascades\lbpcascade_frontalface.xml')
 
# iterating through each image in the dataset and applying the function detect_faces
 
for imagePath in imagePaths: 
    image = cv2.imread(imagePath) # load image
    faces_detected_img = detect_faces(lbp_face_cascade, image) # calling detect_faces function on each image
    cv2.imwrite(imagePath,faces_detected_img) # store detected image    
    cv2.waitKey(0)
 
cv2.destroyAllWindows()

Feature extraction from detected faces - Local Binary Pattern (LBP) algorithm

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def get_pixel(img, center, x, y):
      
  new_value = 0
      
  try:
      # If local neighbourhood pixel 
      # value is greater than or equal
      # to center pixel values then 
      # set it to 1
      if img[x][y] >= center:
            new_value = 1
              
  except:
      # Exception is required when 
      # neighbourhood value of a center
      # pixel value is null i.e. values
      # present at boundaries.
      pass
      
  return new_value

# Function for calculating LBP
def lbp_calculated_pixel(img, x, y):
   
  center = img[x][y]
   
  val_ar = []
      
  # top_left
  val_ar.append(get_pixel(img, center, x-1, y-1))
      
  # top
  val_ar.append(get_pixel(img, center, x-1, y))
      
  # top_right
  val_ar.append(get_pixel(img, center, x-1, y + 1))
      
  # right
  val_ar.append(get_pixel(img, center, x, y + 1))
      
  # bottom_right
  val_ar.append(get_pixel(img, center, x + 1, y + 1))
      
  # bottom
  val_ar.append(get_pixel(img, center, x + 1, y))
      
  # bottom_left
  val_ar.append(get_pixel(img, center, x + 1, y-1))
      
  # left
  val_ar.append(get_pixel(img, center, x, y-1))
       
  # Now, we need to convert binary values to decimal
  power_val = [1, 2, 4, 8, 16, 32, 64, 128]
   
  val = 0
      
  for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
          
  return val

for imagePath in imagePaths:

  img_bgr = cv2.imread(imagePath, 1)
   
  height, width, _ = img_bgr.shape
  
  # We need to convert RGB image into gray one because gray image has one channel only
  img_gray = cv2.cvtColor(img_bgr,cv2.COLOR_BGR2GRAY)
  
  # Create a numpy array as the same height and width of RGB image
  img_lbp = np.zeros((height, width),np.uint8)
  
  for i in range(0, height):
    for j in range(0, width):
      img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
  
  
    cv2.imwrite(imagePath,img_lbp) 
    cv2.waitKey(0)
cv2.destroyAllWindows() 